## Feature Engineering

This is my df that I have scraped from Boxofficemojo. I chose to scrape the top 200 (domestic) movies from 2010-2021. Any titles with "re-release" were omitted from my scrape. My initial database had 2200 titles in it, many of them had missing or null values.

The initial part of my cleaning process took place in DBbrowser, I performed the following:

- All columns with multiple null values were removed in DBbrowser

- I also chose to group the dates into their respective quarters. 
      Q1:JAN-MAR  Q2:APR-JUN  Q3:JUL-SEP  Q4:OCT-DEC 
- Removed all movies with a budget below 10M

After performing the initial cleaning, we can see that I have 1195 columns remaining.

There are still a few issues with this dataframe that need to be addressed before modeling:

- Some of the titles have duplicate entries

- All of my monetary figures are listed as string objects, they also have commas and a dollar sign attached

- Runtime is also a string object, with "hr" and "min" attached. changing these to ints and getting a sum in minutes will make them easier to handle. 

- MPAA rating is a categorical feature, I am planning on fixing this by using dummy variables.

- There are many problems associated with my Genre column, a full overview of these problems can be seen below


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns 
import numpy as np 
import warnings 
warnings.filterwarnings('ignore')

base_df = pd.read_csv('final_boxoffice_data_qdates.csv')



In [3]:
base_df.head(5) #checking to see if everything is here, we can see that our revenue columns still contain $ and commas


,Title,Domestic_Revenue,Worldwide_Revenue,Distributor,Opening_weekend_box,no_Theaters_open,Budget,Release Quarter,MPAA_Rating,Run_Time,Genre,Days_In_theaters
0,12 Years a Slave,"$56,671,993","$187,733,202",Fox Searchlight Pictures,"$923,715",19,"$20,000,000",4,R,2 hr 14 min,"Biography,Drama,History",440
1,127 Hours,"$18,335,230","$60,738,797",Fox Searchlight Pictures,"$264,851",4,"$18,000,000",4,R,1 hr 34 min,"Biography,Drama",422
2,13 Hours,"$52,853,219","$69,411,370",Paramount Pictures,"$16,194,738",2389,"$50,000,000",1,R,2 hr 24 min,"Action,Drama,Thriller,War",352
3,1917,"$159,227,644","$384,983,987",Universal Pictures,"$576,216",11,"$95,000,000",4,R,1 hr 59 min,"Action,Drama,War",373
4,2 Guns,"$75,612,460","$131,940,411",Universal Pictures,"$27,059,130",3025,"$61,000,000",3,R,1 hr 49 min,"Action,Comedy,Crime,Thriller",152


In [4]:
base_df.info()#note that our revenue columns are listed as objects instead of ints

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195 entries, 0 to 1194
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                1195 non-null   object
 1   Domestic_Revenue     1195 non-null   object
 2   Worldwide_Revenue    1195 non-null   object
 3   Distributor          1195 non-null   object
 4   Opening_weekend_box  1195 non-null   object
 5   no_Theaters_open     1195 non-null   int64 
 6   Budget               1195 non-null   object
 7   Release Quarter      1195 non-null   int64 
 8   MPAA_Rating          1195 non-null   object
 9   Run_Time             1195 non-null   object
 10  Genre                1195 non-null   object
 11  Days_In_theaters     1195 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 112.2+ KB


In [5]:
base_df.shape #checking to see the total number of movies represented by our rows

(1195, 12)

In [6]:
base_df[base_df.columns[1:3]] = base_df[base_df.columns[1:3]].replace('[\$,]', '', regex=True).astype(int)
base_df[base_df.columns[4:7]] = base_df[base_df.columns[4:7]].replace('[\$,]', '', regex=True).astype(int)
#this will strip the dollar signs and commas, as well as turn our strings into integer values

In [7]:
base_df.head()#quick check to see if this worked 

,Title,Domestic_Revenue,Worldwide_Revenue,Distributor,Opening_weekend_box,no_Theaters_open,Budget,Release Quarter,MPAA_Rating,Run_Time,Genre,Days_In_theaters
0,12 Years a Slave,56671993,187733202,Fox Searchlight Pictures,923715,19,20000000,4,R,2 hr 14 min,"Biography,Drama,History",440
1,127 Hours,18335230,60738797,Fox Searchlight Pictures,264851,4,18000000,4,R,1 hr 34 min,"Biography,Drama",422
2,13 Hours,52853219,69411370,Paramount Pictures,16194738,2389,50000000,1,R,2 hr 24 min,"Action,Drama,Thriller,War",352
3,1917,159227644,384983987,Universal Pictures,576216,11,95000000,4,R,1 hr 59 min,"Action,Drama,War",373
4,2 Guns,75612460,131940411,Universal Pictures,27059130,3025,61000000,3,R,1 hr 49 min,"Action,Comedy,Crime,Thriller",152


In [8]:
df_2 = base_df.drop_duplicates('Title') # I noticed in SQL that there were duplicate titles, this should drop the extras

In [9]:
df_2.shape #quick check again, looks like it dropped 39 titles

(1156, 12)

In [10]:
#runtime is currently listed as a string object, it also has 'hr' and 'min' still attached, lets strip those and 
#combine in a new column. Also, if anyone is reading this, please let me know if there is an easier way to do this
#I spent way too long on it. lol
df_2['hour'] = (df_2['Run_Time'].astype(str).str[0]).astype(int)
df_2['min'] = np.where(df_2['Run_Time'].str.contains('min'), ((df_2['Run_Time'].astype(str).str[5:]).str.replace('min', '')), 0).astype(int)
df_2['Running_Time(min)'] = (df_2['hour'].astype(int)*60)+df_2['min']

# I don't want to keep these new columns. I also have no need for my original Run_Time
del df_2['hour']
del df_2['min']
del df_2['Run_Time']

#this kicked back a warning, since everything is still functional I decided to mute it

In [11]:
df_2.head() #We're getting there!

,Title,Domestic_Revenue,Worldwide_Revenue,Distributor,Opening_weekend_box,no_Theaters_open,Budget,Release Quarter,MPAA_Rating,Genre,Days_In_theaters,Running_Time(min)
0,12 Years a Slave,56671993,187733202,Fox Searchlight Pictures,923715,19,20000000,4,R,"Biography,Drama,History",440,134
1,127 Hours,18335230,60738797,Fox Searchlight Pictures,264851,4,18000000,4,R,"Biography,Drama",422,94
2,13 Hours,52853219,69411370,Paramount Pictures,16194738,2389,50000000,1,R,"Action,Drama,Thriller,War",352,144
3,1917,159227644,384983987,Universal Pictures,576216,11,95000000,4,R,"Action,Drama,War",373,119
4,2 Guns,75612460,131940411,Universal Pictures,27059130,3025,61000000,3,R,"Action,Comedy,Crime,Thriller",152,109


In [12]:
#now we just have to assign dummy vars to our MPAA Rating

df_2 = pd.concat([df_2.drop('MPAA_Rating', axis=1), pd.get_dummies(df_2['MPAA_Rating'])], axis=1)

#I don't want the dash in PG-13 
df_2 = df_2.rename(columns = {'PG-13':'PG_13'})

In [13]:
#I also discovered release date did not work well with a numeric representation of quarter, so i changed them to dummies
df_2 = pd.concat([df_2.drop('Release Quarter', axis=1), pd.get_dummies(df_2['Release Quarter'])], axis=1)

#I don't want the dash in PG-13 
df_2 = df_2.rename(columns = {1 :'winter', 2 : 'spring', 3 : 'summer', 4 : 'fall_holiday'})

In [14]:
df_2.head()

,Title,Domestic_Revenue,Worldwide_Revenue,Distributor,Opening_weekend_box,no_Theaters_open,Budget,Genre,Days_In_theaters,Running_Time(min),G,PG,PG_13,R,winter,spring,summer,fall_holiday
0,12 Years a Slave,56671993,187733202,Fox Searchlight Pictures,923715,19,20000000,"Biography,Drama,History",440,134,0,0,0,1,0,0,0,1
1,127 Hours,18335230,60738797,Fox Searchlight Pictures,264851,4,18000000,"Biography,Drama",422,94,0,0,0,1,0,0,0,1
2,13 Hours,52853219,69411370,Paramount Pictures,16194738,2389,50000000,"Action,Drama,Thriller,War",352,144,0,0,0,1,1,0,0,0
3,1917,159227644,384983987,Universal Pictures,576216,11,95000000,"Action,Drama,War",373,119,0,0,0,1,0,0,0,1
4,2 Guns,75612460,131940411,Universal Pictures,27059130,3025,61000000,"Action,Comedy,Crime,Thriller",152,109,0,0,0,1,0,0,1,0


## The Genre Problem
Although I originally intended for Genre to be one of my final features, I have run into a few issues with it. 
Boxofficemojo allows for multiple genre assignment for each film. This allows for a very high number of unique 
genre combinations for each movie. Some films have as many as 10 genres attached to their page. When I tried to assign a numeric value for genres, the values ranged from 0-400 with very few movies sharing a unique id. When
I visualized this in a scatter plot, the result was too random. Since my deadline is approaching, I have decided 
to omit genre from my final targets. 

In [15]:
#The Genre Column is looking like it might be a bit tricky as there are multiple categorical values with commas 
#separating them
genres = np.unique(', '.join(df_2['Genre']).split(', '))#this will separate out our unique combinations of genres, lets encode each one with a number
genres

array(['Action',
       'Action,Adventure,Animation,Comedy,Crime,Family,Fantasy,Romance,Sci-Fi',
       'Action,Adventure,Animation,Comedy,Crime,Family,Romance,Sci-Fi,Thriller',
       'Action,Adventure,Animation,Comedy,Drama,Family,Fantasy',
       'Action,Adventure,Animation,Comedy,Drama,Family,Fantasy,Romance',
       'Action,Adventure,Animation,Comedy,Drama,Family,Sci-Fi',
       'Action,Adventure,Animation,Comedy,Family',
       'Action,Adventure,Animation,Comedy,Family,Fantasy',
       'Action,Adventure,Animation,Comedy,Family,Fantasy,Musical,Sci-Fi',
       'Action,Adventure,Animation,Comedy,Family,Sci-Fi',
       'Action,Adventure,Animation,Drama,Family,Fantasy',
       'Action,Adventure,Animation,Family,Fantasy',
       'Action,Adventure,Animation,Family,Fantasy,Mystery,Thriller,War',
       'Action,Adventure,Animation,Family,Fantasy,Sci-Fi',
       'Action,Adventure,Animation,Family,Sci-Fi',
       'Action,Adventure,Biography,Drama',
       'Action,Adventure,Biography,Drama,H

In [16]:
from sklearn.preprocessing import LabelEncoder


In [17]:
gle = LabelEncoder()
genre_labels = gle.fit_transform(df_2['Genre'])
genre_mappings = {index: label for index, label in 
                  enumerate(gle.classes_)}
#this will give each combination of genres its own unique numeric label
#there are currently too many combinations of genres.

In [18]:
df_2['Genre Label'] = genre_labels

In [19]:
df_2.columns #looks like we have a bit of trailing whitespace

Index(['Title', 'Domestic_Revenue', 'Worldwide_Revenue', 'Distributor',
       'Opening_weekend_box', 'no_Theaters_open', 'Budget', 'Genre',
       'Days_In_theaters', 'Running_Time(min)', 'G', 'PG', 'PG_13', 'R',
       'winter', 'spring', 'summer', 'fall_holiday', 'Genre Label'],
      dtype='object')

In [20]:
df_2.columns = df_2.columns.map(str.strip)
df_2.columns

Index(['Title', 'Domestic_Revenue', 'Worldwide_Revenue', 'Distributor',
       'Opening_weekend_box', 'no_Theaters_open', 'Budget', 'Genre',
       'Days_In_theaters', 'Running_Time(min)', 'G', 'PG', 'PG_13', 'R',
       'winter', 'spring', 'summer', 'fall_holiday', 'Genre Label'],
      dtype='object')

In [21]:
#lets save this so that we can work on it in our modeling folder
df_2

,Title,Domestic_Revenue,Worldwide_Revenue,Distributor,Opening_weekend_box,no_Theaters_open,Budget,Genre,Days_In_theaters,Running_Time(min),G,PG,PG_13,R,winter,spring,summer,fall_holiday,Genre Label
0,12 Years a Slave,56671993,187733202,Fox Searchlight Pictures,923715,19,20000000,"Biography,Drama,History",440,134,0,0,0,1,0,0,0,1,241
1,127 Hours,18335230,60738797,Fox Searchlight Pictures,264851,4,18000000,"Biography,Drama",422,94,0,0,0,1,0,0,0,1,235
2,13 Hours,52853219,69411370,Paramount Pictures,16194738,2389,50000000,"Action,Drama,Thriller,War",352,144,0,0,0,1,1,0,0,0,132
3,1917,159227644,384983987,Universal Pictures,576216,11,95000000,"Action,Drama,War",373,119,0,0,0,1,0,0,0,1,133
4,2 Guns,75612460,131940411,Universal Pictures,27059130,3025,61000000,"Action,Comedy,Crime,Thriller",152,109,0,0,0,1,0,0,1,0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,Your Highness,21596445,28013733,Universal Pictures,9360020,2769,49900000,"Adventure,Comedy,Fantasy,Romance",268,102,0,0,0,1,0,1,0,0,199
1191,Youth in Revolt,15281286,19651093,Dimension Films,6888334,1873,18000000,"Comedy,Drama,Romance",358,90,0,0,0,1,1,0,0,0,280
1192,Zero Dark Thirty,95720716,132820716,Sony Pictures Entertainment (SPE),417150,5,40000000,"Drama,History,Thriller,War",378,157,0,0,0,1,0,0,0,1,346
1193,Zombieland: Double Tap,73123082,122810399,Sony Pictures Entertainment (SPE),26803104,3468,42000000,"Action,Comedy,Horror",441,99,0,0,0,1,0,0,0,1,97


In [101]:
df_2.to_csv('movie_features.csv')